In [ ]:
!wget https://raw.githubusercontent.com/happylolonly/events-free-ml/master/tags/data/data.csv

In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz

In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz

In [ ]:
# !gunzip cc.ru.300.bin.gz
# !gunzip cc.ru.300.vec.gz

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/happylol/Desktop/projects/ml/events-free-ml/tags/data/data.csv')
df.head()

df['tags'] = df['tags'].map(eval) 
# tags_white_list = {'митап', 'музыка', 'тренировка', 'дети', 'вечеринка'}
tags_white_list = {'выставка', 'праздник', 'фитнес', 'тренировка', 'работа', 'вебинар', 'германия', 'trx', 'english', 'лекция', 'обучение', 'музыка', 'джаз', 'форро', 'танцы', 'it', 'митап', 'дизайн', 'семинар', 'концерт', 'конкурс', 'вечеринка', 'ярмарка', 'тенис', 'турнир', 'театр', 'фотография', 'фильм', 'кино', 'сальса', 'дети', 'гитара', 'стартап', 'инвестиции', 'маркет', 'чемпионат', 'программирование', 'нетворкинг', 'право', 'день рождения', 'презентация', 'блокчейн', 'финтех', 'тестирование'}
# tags_white_list = {'выставка', 'фитнес', 'праздник', 'тренировка', 'лекция', 'обучение', 'музыка', 'it', 'митап', 'вечеринка', 'ярмарка', 'дети'}
# tags_white_list = {'фитнес', 'праздник', 'тренировка', 'обучение', 'музыка', 'митап', 'вечеринка', 'ярмарка', 'дети'}
# df['tags'] = df['tags'].map(lambda x: set(x))
df['tags'] = df['tags'].map(lambda x: set(x).intersection(tags_white_list))
# .intersection(tags_white_list)
df = df[df.tags.map(len) > 0]

df.head()
df.shape

(154, 3)

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Я0-9a-zA-Z]", " ")
    df[text_field] = df[text_field].str.replace(r"\s+", " ")
    df[text_field] = df[text_field].str.lower()
    return df

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/happylol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = standardize_text(df, 'text')
df = standardize_text(df, 'title')

df.head(20)

,title,text,tags
0,письма из венеции,никто не может остаться равнодушным к венеции ...,{выставка}
1,а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...,"{фитнес, праздник}"
2,poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...,"{тренировка, фитнес}"
3,бесплатная открытая тренировка по crossfit n ...,бесплатная открытая тренировка по crossfit n n...,"{тренировка, фитнес}"
4,хотите работать в германии но не владеете дос...,хотите работать в германии но не владеете дос...,"{германия, работа, вебинар}"
5,хотите работать в германии но не владеете дос...,хотите работать в германии но не владеете дос...,"{германия, работа, вебинар}"
6,внимание n nоткрытое занятие по trx и super 8...,внимание n nоткрытое занятие по trx и super 8 ...,"{тренировка, фитнес, trx}"
7,открытое занятие по trx и super 8 а так же на...,открытое занятие по trx и super 8 а так же на...,"{тренировка, фитнес, trx}"
8,демонстрационное занятие по английскому n nес...,демонстрационное занятие по английскому n nесл...,"{лекция, обучение, english}"
9,открытый урок nфитнес центр lifestyle fitness...,открытый урок nфитнес центр lifestyle fitness...,"{тренировка, фитнес}"


In [5]:
from collections import Counter

counter = Counter()
for tags in df['tags'].values:
    for t in tags:
        counter[t] += 1

In [6]:
counter

Counter({'выставка': 7,
         'фитнес': 17,
         'праздник': 14,
         'тренировка': 16,
         'германия': 2,
         'работа': 2,
         'вебинар': 2,
         'trx': 3,
         'лекция': 8,
         'обучение': 11,
         'english': 5,
         'джаз': 5,
         'музыка': 36,
         'танцы': 4,
         'форро': 2,
         'it': 7,
         'митап': 25,
         'дизайн': 3,
         'семинар': 3,
         'концерт': 3,
         'конкурс': 3,
         'вечеринка': 20,
         'ярмарка': 16,
         'турнир': 4,
         'тенис': 4,
         'театр': 6,
         'фотография': 3,
         'кино': 4,
         'фильм': 4,
         'сальса': 2,
         'дети': 15,
         'гитара': 3,
         'стартап': 5,
         'инвестиции': 4,
         'маркет': 4,
         'чемпионат': 2,
         'программирование': 3,
         'право': 2,
         'нетворкинг': 4,
         'день рождения': 2,
         'презентация': 2,
         'блокчейн': 2,
         'финтех': 2,
    

In [7]:
rows = []

for row in df.to_dict(orient='record'):
    for tag in row['tags']:
        row_copy = row.copy()
        row_copy['tag'] = tag
        
        rows.append(row_copy)
        
df = pd.DataFrame(rows)
df.head()

,tag,tags,text,title
0,выставка,{выставка},никто не может остаться равнодушным к венеции ...,письма из венеции
1,фитнес,"{фитнес, праздник}",а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...
2,праздник,"{фитнес, праздник}",а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...
3,тренировка,"{тренировка, фитнес}",poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...
4,фитнес,"{тренировка, фитнес}",poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...


In [8]:
from sklearn.model_selection import train_test_split

newDf = df.copy();


arr = []

train, test = train_test_split(df, test_size=0.3,  stratify=df['tag'].values)
for i in counter:
    if counter[i] > 2:
        arr.append(i);
        
        
#         print(newDf[newDf['tags'] == i].values)


type(newDf['tags'][0])
print(arr);
print(np.unique(train['tag'].values, return_counts=True))
print(np.unique(test['tag'].values, return_counts=True))

['выставка', 'фитнес', 'праздник', 'тренировка', 'trx', 'лекция', 'обучение', 'english', 'джаз', 'музыка', 'танцы', 'it', 'митап', 'дизайн', 'семинар', 'концерт', 'конкурс', 'вечеринка', 'ярмарка', 'турнир', 'тенис', 'театр', 'фотография', 'кино', 'фильм', 'дети', 'гитара', 'стартап', 'инвестиции', 'маркет', 'программирование', 'нетворкинг']
(array(['english', 'it', 'trx', 'блокчейн', 'вебинар', 'вечеринка',
       'выставка', 'германия', 'гитара', 'день рождения', 'дети', 'джаз',
       'дизайн', 'инвестиции', 'кино', 'конкурс', 'концерт', 'лекция',
       'маркет', 'митап', 'музыка', 'нетворкинг', 'обучение', 'право',
       'праздник', 'презентация', 'программирование', 'работа', 'сальса',
       'семинар', 'стартап', 'танцы', 'театр', 'тенис', 'тестирование',
       'тренировка', 'турнир', 'фильм', 'финтех', 'фитнес', 'форро',
       'фотография', 'чемпионат', 'ярмарка'], dtype=object), array([ 4,  5,  2,  1,  1, 14,  5,  1,  2,  1, 11,  4,  2,  3,  3,  2,  2,
        6,  3, 18, 25

In [9]:
# %%bash

# pip install -U -q git+https://github.com/facebookresearch/fastText.git
# import fastText

import fastText

# fasttext?

FIELD = 'text'

# print(df)

In [10]:
def save(df, path):
    with open(path, 'w+') as f:
        for _, row in df.iterrows():
            f.write('__label__{} {}\n'.format(row['tag'], row[FIELD]))

In [11]:
save(df, '/tmp/train.data')
# save(train, '/tmp/train.data')
# train.shape
# test.shape

In [12]:
model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors='cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)

In [13]:
# print(test.shape)
labels, probs = model.predict(test[FIELD].tolist())
# print(labels, probs)
labels = [ll[0].replace('__label__', '') for ll in labels]
print(labels)

['фитнес', 'лекция', 'митап', 'вебинар', 'митап', 'митап', 'обучение', 'музыка', 'танцы', 'вечеринка', 'ярмарка', 'концерт', 'музыка', 'митап', 'нетворкинг', 'тренировка', 'митап', 'фитнес', 'english', 'фитнес', 'trx', 'trx', 'кино', 'дети', 'музыка', 'программирование', 'музыка', 'музыка', 'день', 'обучение', 'музыка', 'музыка', 'обучение', 'музыка', 'праздник', 'праздник', 'ярмарка', 'дизайн', 'танцы', 'митап', 'финтех', 'фитнес', 'праздник', 'кино', 'дети', 'программирование', 'музыка', 'программирование', 'дети', 'митап', 'лекция', 'ярмарка', 'музыка', 'лекция', 'чемпионат', 'вебинар', 'музыка', 'ярмарка', 'музыка', 'выставка', 'митап', 'музыка', 'trx', 'музыка', 'праздник', 'дизайн', 'митап', 'вечеринка', 'выставка', 'дети', 'стартап', 'митап', 'маркет', 'митап', 'праздник', 'музыка', 'тренировка', 'ярмарка', 'тенис', 'english', 'праздник', 'тенис', 'музыка', 'музыка', 'фитнес', 'вебинар', 'english', 'ярмарка']


In [14]:
from sklearn.metrics import classification_report
# from apihelper import info
# dir(model)

In [ ]:
print(classification_report(test['tag'].values, labels))
# test[FIELD].tolist()
test['tag'].values

import numpy as np

# n = 3
# top_n_predictions = np.argsort(probs, axis=1)[:, -n:]
# top_n_predictions
# probs

In [ ]:
# test[FIELD].tolist()

# fasttext
# model = fasttext.load_model('model.bin')
# model.save_model('ft.model')
#('./model.bin')
# print(model2.words) # list of words in dictionary
# print(model2['king']) # get the vector of the word 'king'

labels, probs = model2.predict(['n расписание 24 декабря 11 00 15 0025 декабря 11 00 15 00за неделю до нового года любимый дедушка мороз решил собрать всех всех своих друзей в софатэкс повстречаться с дедом морозом перед самым волшебным днем в году можно будет в самом уютном месте магазине чехлов для мебели софатэкс где тепло и по домашнему спокойно спокойно но только для взрослых детки будут радостно кружиться с дедом морозом в хороводе рассказывать ему стихотворения и конечно же свои заветные желания и пока дети веселятся с дедом морозом родители могут выбрать подарочки себе и своим близким все знают как сложно выбраться деду морозу из своей мастерской прямо перед новым годом но он сделает исключение и проведет время со своими друзьями ты же его друг верно тогда приходи 24 и 25 декабря в софатэкс но только не проспи сани прилетают ровно в 11 дед мороз не отпустит никого с пустыми руками каждому вручит подарок выслушает каждое желание но взамен нужно будет порадовать его стишком дедушка очень любит стихотворения приходи и бери своих друзей место встречи с дедом морозом магазин софатэкс в тц зеркало ул в хоружей 6 вход с улицы розовая вывеска софатэкс добраться можно автобусы 19 24 25 29 44 59 91 136 троллейбусы 12 22 29 40 до остановки комаровский рыноклибо трамваи 1 5 6 11автобусы 100 115эметро до остановки пл якуба колоса магазин софатэкс г минск ул веры хоружей 624 25 декабря 2018г с 11 00 по 15 00вход свободный любую информацию можно получить по телефону 375 33 333 00 66 или на сайте sofatex by n n n n t t t t t t n '])
print(labels)


In [15]:
model.quantize(qnorm=True, cutoff=100000)
model.save_model('ft_quantized.6.model')

In [ ]:
# model2 = 
model2 = fastText.load_model('ft_quantized.model')


In [ ]:
labels, probs = model2.predict_proba(test[FIELD].tolist())
labels = [ll[0].replace('__label__', '') for ll in labels]
print(classification_report(test['tag'].values, labels))